## Libraries

In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
import geopandas as gdp
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import re

from src import CleaningFunctions as cf

## Load Data 

In [3]:
barcelona = pd.read_csv('../Data/GeoDataFrames/gdf_barcelona.csv')
madrid = pd.read_csv('../Data/GeoDataFrames/gdf_madrid.csv')
gijon = pd.read_csv('../Data/GeoDataFrames/gdf_gijon.csv')
bilbao = pd.read_csv('../Data/GeoDataFrames/gdf_bilbao.csv')

## Cleaning dataframes

The category column had inverted quotes. I used the replace to remove them and the strip function to remove the spaces between the words. 

In [4]:
df_madrid = cf.cleaning(madrid, 'category', 'madrid')

../src/CleaningFunctions.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r"[\"\',]", '')


In [5]:
df_barcelona = cf.cleaning(barcelona, 'category', 'barcelona')

../src/CleaningFunctions.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r"[\"\',]", '')


In [6]:
df_gijon = cf.cleaning(gijon, 'category', 'gijon')

../src/CleaningFunctions.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r"[\"\',]", '')


In [7]:
df_bilbao = cf.cleaning(bilbao, 'category', 'bilbao')

../src/CleaningFunctions.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r"[\"\',]", '')


I put the two dataframes together in order to make the ranking.

In [8]:
data =df_madrid.append([df_barcelona, df_gijon, df_bilbao])

In [9]:
total_data = data.groupby(["category", "city"]).mean()

In [10]:
total_data.reset_index(drop = False)

,category,city,lat,long,distance
0,Bar,barcelona,41.392523,2.165113,2.153021
1,Bar,bilbao,43.261339,-2.933254,0.874563
2,Bar,gijon,43.537687,-5.658454,5.504250
3,Bar,madrid,40.427318,-3.677368,0.986381
4,Bars,barcelona,41.396544,2.155763,0.854449
5,Basketball Court,barcelona,41.405313,2.168945,2.584342
6,Basketball Court,bilbao,43.260186,-2.948913,2.337619
7,Basketball Court,gijon,43.447403,-5.762851,15.403770
8,Basketball Court,madrid,40.423556,-3.674405,2.173045
9,Preschool,barcelona,41.399940,2.163414,2.955602


In [12]:
df_min_max_scaled = cf.normalization(total_data, 'distance')

In [13]:
df_min_max_scaled['Weights'] = df_min_max_scaled.apply(cf.weights, axis=1)

In [14]:
df_min_max_scaled["Rank"] = df_min_max_scaled["distance"] * df_min_max_scaled["Weights"]


In [15]:
total_data = df_min_max_scaled.groupby(["city"]).sum()

In [16]:
total_data.sort_values("Rank")

,lat,long,distance,Weights,Rank
city,,,,,
bilbao,173.040000,-11.739815,0.111485,0.9,0.016615
barcelona,248.394706,12.959131,0.320070,1.2,0.051966
madrid,202.098004,-18.410392,0.339527,1.0,0.060553
gijon,217.390586,-28.718730,2.532132,1.0,0.383190
